# Split the Data 
#### Splitting the date as : 
- 60% Train set 
- 20% Validation set 
- 20% Test Set 

In [1]:
import pandas as pd 




In [2]:
def remove_unamed(dfs):
    removed = []
    for df in dfs:
        removed.append(df.loc[:, ~df.columns.str.contains('^Unnamed')])
        


fallen_Trees = 'fallen_Trees'
uncollected_waste = 'uncollected_waste'

def map_distortion(value):
    
    if value == 0:
        return 'allocated_waste'
    
    return 'fallen_trees'
    


In [3]:
df = pd.read_csv('annotation.csv')

df.columns

Index(['Unnamed: 0', 'folder_name', 'image_url', 'class_name', 'x1', 'y1', '5',
       '6', 'x2', 'y2', '9', '10'],
      dtype='object')

In [4]:
df.class_name.value_counts()

0    533
1    258
Name: class_name, dtype: int64

In [5]:
df.query('class_name == "fallen_Trees"')

,Unnamed: 0,folder_name,image_url,class_name,x1,y1,5,6,x2,y2,9,10


In [6]:
df.class_name = df.class_name.apply(lambda x : uncollected_waste if x == 0 else fallen_Trees)


In [7]:
df.class_name.unique()

array(['uncollected_waste', 'fallen_Trees'], dtype=object)

In [8]:
# df.query('class_name == 0')

In [9]:
df.shape

(791, 12)

In [10]:
val_set = df.sample(frac=0.2,random_state=1337)
val_set.folder_name = 'VALIDATE'
df.drop(val_set.index,inplace=True)
val_set = val_set.loc[:, ~val_set.columns.str.contains('^Unnamed')]
val_set.shape
val_set.head()
val_set.shape

(158, 11)

In [11]:
test_set = df.sample(frac=0.2,random_state=1337)
test_set.folder_name = 'TEST'
test_set = test_set.loc[:, ~test_set.columns.str.contains('^Unnamed')]
df.drop(test_set.index,inplace=True)
test_set.shape

(127, 11)

In [12]:
train_set = df
train_set.folder_name = 'TRAIN'
train_set = train_set.loc[:, ~train_set.columns.str.contains('^Unnamed')]
train_set.shape


(506, 11)

In [13]:
df = pd.concat([test_set,val_set,train_set])

In [14]:
df.folder_name.unique()

array(['TEST', 'VALIDATE', 'TRAIN'], dtype=object)

In [15]:
df

,folder_name,image_url,class_name,x1,y1,5,6,x2,y2,9,10
260,TEST,distortion_new/20210905_170449.jpg,uncollected_waste,0.615722,0.071729,NaN,NaN,0.444925,0.017638,NaN,NaN
258,TEST,distortion_new/20210905_170705.jpg,uncollected_waste,0.697706,0.142602,NaN,NaN,0.492037,0.038944,NaN,NaN
242,TEST,distortion_new/20210905_165543.jpg,fallen_Trees,0.413043,0.310315,NaN,NaN,0.431586,0.170077,NaN,NaN
657,TEST,distortion_new/20210905_165609.jpg,uncollected_waste,0.048167,0.078431,NaN,NaN,0.445652,0.031969,NaN,NaN
143,TEST,distortion_new/20210905_170343.jpg,uncollected_waste,0.767690,0.170503,NaN,NaN,0.519821,0.101023,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
783,TRAIN,distortion_new/20210905_170733.jpg,fallen_Trees,0.592072,0.129582,NaN,NaN,0.315217,0.095908,NaN,NaN
784,TRAIN,distortion_new/20210905_170733.jpg,fallen_Trees,0.816496,0.367008,NaN,NaN,0.421355,0.223785,NaN,NaN
786,TRAIN,distortion_new/20210905_165551.jpg,fallen_Trees,0.143436,0.286871,NaN,NaN,0.402813,0.212276,NaN,NaN
787,TRAIN,distortion_new/20210905_165551.jpg,fallen_Trees,0.408781,0.537084,NaN,NaN,0.512788,0.340153,NaN,NaN


In [16]:
df.to_csv('cleared.csv', header=False, index=False)

In [17]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf

In [18]:
tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [19]:
spec = model_spec.get('efficientdet_lite0')


In [20]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('cleared.csv')


In [21]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
23/23 [==============================] - 118s 3s/step - det_loss: 25.3761 - cls_loss: 1.0196 - box_loss: 0.4871 - reg_l2_loss: 0.0631 - loss: 25.4393 - learning_rate: 0.0090 - gradient_norm: 8.7184 - val_det_loss: 37.0931 - val_cls_loss: 0.7360 - val_box_loss: 0.7271 - val_reg_l2_loss: 0.0634 - val_loss: 37.1565
Epoch 2/50
23/23 [==============================] - 64s 3s/step - det_loss: 15.0091 - cls_loss: 0.5177 - box_loss: 0.2898 - reg_l2_loss: 0.0636 - loss: 15.0727 - learning_rate: 0.0100 - gradient_norm: 10.0000 - val_det_loss: 17.9586 - val_cls_loss: 0.3141 - val_box_loss: 0.3529 - val_reg_l2_loss: 0.0637 - val_loss: 18.0223
Epoch 3/50
23/23 [==============================] - 61s 3s/step - det_loss: 14.8563 - cls_loss: 0.3130 - box_loss: 0.2909 - reg_l2_loss: 0.0637 - loss: 14.9201 - learning_rate: 0.0099 - gradient_norm: 10.0000 - val_det_loss: 20.4789 - val_cls_loss: 0.2248 - val_box_loss: 0.4051 - val_reg_l2_loss: 0.0638 - val_loss: 20.5426
Epoch 4/50
23/23 [=======